### 목적 : 미세먼지봇 구현. 
1. 사용자의 텍스트 입력 ex)상도역, 강남 교보문고
2. 구글 API에서 지역 이름 출력(도(광역시), 시, 동(면))
3. 동 이름을 이용하여 환경공단 API에 대입, 최단거리의 관측소 좌표 크롤링
4. 환경공단 API에서 해당 지역 데이터를 제공하지 못하는경우
> 지역이름을 벡터화, 코사인 유사도로 지형적으로 가까운 근처 동 이름 제공
> 관측소 좌표 크롤링  

5. 환경공단 API에서 해당 지역 데이터를 제공하는 경우
> 관측소 좌표 크롤링

6. 관측소를 기준으로 측정한 최신 미세먼지 데이터 크롤링
7. 사용자에게 return


# 4번 단계 구현을 위한 word2vec 과정

####  word2vec의 CBOW
- "비슷한 단어는 같은 문맥에 사용된다"
- text 파일 사용, 성능 향상을 위한 특수 기호 및 영어의 경우 stemming 사용
- 이후 tokenization 수행, 이 때 tokenization은 string -> list를 의미.  
ex) [['token', 'token'], ['token', 'token']]

Codec이란?
- 파이썬에서 일반적으로 유니코드를 다룰 때 사용하는 decode, encode는 대용량 데이터인 경우, 모든 데이터가 메모리에 올라감
- ** codecs ** 모듈을 통해 파일을 조금씩 읽어, 메모리에도 약간의 데이터만 올려줄 수 있음

In [3]:
import codecs
from gensim.models import Word2Vec
import multiprocessing

import pandas as pd
import numpy as np
import glob

In [1]:
# 주소가 있는 csv 파일들을 불러와 word2vec에 넣기 위한 전처리하는 함수
# 큰 단위의 주소와 세부 단위 주소 따로 합쳐져 있는 파일을 동일 value를 가지는 column을 기준으로 merge
def merge_and_to_csv(add_fname, sub_fname) : 
    address_lines = open("datas/" + add_fname + ".txt", "r")
    sub_lines = open("datas/" + sub_fname + ".txt", "r")
    
    address_list, sub_list = [], []
    
    for add_line in address_lines : 
        add_line = (add_line).split("|")
        address_list.append(add_line)
    
    for sub_line in sub_lines : 
        sub_line = sub_line.split("|")
        sub_list.append(sub_line)
        
    address_df = pd.DataFrame(address_list)
    address_df = address_df[[0, 3, 4, 5]]
    
    sub_df = pd.DataFrame(sub_list)
    sub_df = sub_df[[0, 2, 6, 7]]
    
    full_address = pd.merge(address_df, sub_df, how = 'left', on = [0])
    full_address.to_csv("full_" + add_fname + ".csv", index = False)

In [4]:
#작업 폴더 내 파일들 이름으로 리스트 만들기
ls = glob.glob("C:/Users/ledes/study/Microdust_Kakao_chatbot/chatbot/microdust/datas/*csv")

In [10]:
# 큰 단위 주소, 세부 단위 주소를 인덱싱해 리스트에 담기.
ls = ls[ : -1]
file_ls = []
for idx in range(len(ls)) : 
    f_ls = ls[idx][69 :  ]
    f_ls = f_ls[ : -4]
    file_ls.append(f_ls)
    

In [11]:
for address, sub in zip(file_ls[:17], file_ls[17:]) :
    merge_and_to_csv(address, sub)

----

In [6]:
full_address_ls = []
for name in file_ls[:17] : 
    full_name = "full" + name
    full_address_ls.append(full_name)

In [7]:
#word2vec model들의 이름만 
check_ls = glob.glob("C:/Users/ledes/study/Microdust_Kakao_chatbot/chatbot/microdust/model*")

In [8]:
check_ls = [check_ls[i][69:] for i in range(len(check_ls))]
answer_ls = [file_ls[i][13 : -4] for i in range(len(file_ls))]

In [102]:
#for문 과정에서 발생한 오류 확인 -> 작업이 되지 않은 csv 파일들 내용 확인
result = [i for i in answer_ls if i not in check_ls]
result

['sejong', 'seoul', 'ulsan']

In [14]:
file_ls = glob.glob("C:/Users/ledes/study/Microdust_Kakao_chatbot/chatbot/microdust/datas/*csv")
file_ls = [file_ls[i][63: ] for i in range(len(file_ls))]

In [73]:
test = pd.read_csv("full_address_sejong.csv")

In [77]:
test.drop(labels = ["0", "4", "6", "7"], inplace = True, axis = 1, errors = "ignore")

In [71]:
def train_word_vectors(fname) : 
    
    model_name = "_" + fname[13:-4]
    print(model_name + "start")
    #파일 불러오기
    df = pd.read_csv(fname)

    #필요 데이터 추출
    df.drop(labels = ["0", "6", "7"], inplace = True, axis = 1, errors = "ignore")

    #null 값 제거
    df.dropna(axis = 0, how = 'any', inplace = True)

    train_data = []
    for idx in np.array(df) : 
        train_data.append(list(idx))

    #word2vec parameters
    parameters = {
        "sg" : 1, # skip-gram
        "size" : 150, # dimesionality of the feature vectors
        "window" : 3,
        "alpha" : 0.01, #learning rate
        "batch_words" : 10000, #사전 구축 시 한 번에 몇 개의 단어를 읽을 것인가
        "iter" : 10, # epoch과 유사
        "workers" : multiprocessing.cpu_count(), #cpu

    }

    #인수 입력
    model = Word2Vec(**parameters)

    #단어 사전 생성
    model.build_vocab(train_data)

    #벡터 train
    model.train(ttest, total_examples = 100, epochs = 100 )
    
    model.save('model' + model_name)
    
    print(model_name + "end")

In [72]:
for fname in file_ls : 
    train_word_vectors(fname)

_chungbukstart
_chungbukend
_chungnamstart
_chungnamend
_daegustart
_daeguend
_daejeonstart
_daejeonend
_gangwonstart
_gangwonend
_gwangjustart
_gwangjuend
_gyungbukstart
_gyungbukend
_gyunggistart
_gyunggiend
_gyungnamstart
_gyungnamend
_incheonstart
_incheonend
_jejustart
_jejuend
_junbukstart
_junbukend
_junnamstart


C:\Users\ledes\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2802: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


_junnamend
_sejongstart


RuntimeError: you must first build vocabulary before training the model

In [50]:
model.wv.most_similar("동작동", topn = 10)

[('사당동', 0.9160662889480591),
 ('본동', 0.9099898338317871),
 ('신대방동', 0.892453670501709),
 ('상도1동', 0.8730696439743042),
 ('신대방제1동', 0.8628720641136169),
 ('신대방제2동', 0.8610486388206482),
 ('노량진제1동', 0.8286268711090088),
 ('노량진동', 0.8191577196121216),
 ('노량진제2동', 0.798297643661499),
 ('상도제1동', 0.7953025698661804)]

In [51]:
model.save('model' + '_seoul')

In [ ]:
model.wv.similarity("흑석동", "도봉구")

In [ ]:
model.wv.most_similar(positive = ["흑석동", "종로구"], negative = ["상도동"])

#### Comment : 
 word2vec 실습 결과 깨달은 사실
- word2vec 기본은 center word를 바탕으로 주변 context를 판단하거나(skip-gram), 주변 context를 바탕으로 center word를 판단하는(CBOW) 방식이 있음
- 구현해야 하는 기능은 **오타가 없다는 전제 하에, API가 인식하지 못하는 지역명을 바로 잡아주는 것(예를 들어, 충북, 구로5동 등)**
- 해결 아이디어 : 전국 시,도 등 주소가 적힌 csv파일을 찾아 훈련시켜볼 것

갑자기 생각난 아이디어!  
- 오타가 없다는 전제 하에 API가 인식하지 못하는 지역명 바로잡기
> 전국 주소를 word2vec으로 학습
- 이외 데이터 학습은 seq2seq로